In [ ]:
import pandas as pd
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load Titanic dataset from seaborn or URL
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Select relevant features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
df = df[features + ['Survived']]

# Preprocessing
# Fill missing values
imputer = SimpleImputer(strategy='most_frequent')
df[['Age', 'Embarked']] = imputer.fit_transform(df[['Age', 'Embarked']])

# Encode categorical features
le_sex = LabelEncoder()
le_embarked = LabelEncoder()
df['Sex'] = le_sex.fit_transform(df['Sex'])  # male=1, female=0
df['Embarked'] = le_embarked.fit_transform(df['Embarked'])

# Split dataset
X = df[features]
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Prediction function
def predict_survival(Pclass, Sex, Age, SibSp, Parch, Fare, Embarked):
    sex_encoded = le_sex.transform([Sex])[0]
    embarked_encoded = le_embarked.transform([Embarked])[0]
    input_data = pd.DataFrame([[Pclass, sex_encoded, Age, SibSp, Parch, Fare, embarked_encoded]],
                              columns=features)
    prediction = model.predict(input_data)[0]
    return "Survived ✅" if prediction == 1 else "Did Not Survive ❌"

# Gradio interface
interface = gr.Interface(
    fn=predict_survival,
    inputs=[
        gr.Dropdown([1, 2, 3], label="Passenger Class (Pclass)"),
        gr.Radio(['male', 'female'], label="Sex"),
        gr.Slider(0, 80, value=30, label="Age"),
        gr.Slider(0, 5, value=0, label="Siblings/Spouses Aboard (SibSp)"),
        gr.Slider(0, 5, value=0, label="Parents/Children Aboard (Parch)"),
        gr.Slider(0, 500, value=50, label="Fare"),
        gr.Dropdown(['S', 'C', 'Q'], label="Port of Embarkation (Embarked)")
    ],
    outputs="text",
    title="🚢 Titanic Survival Prediction",
    description="Enter passenger details to predict survival using a Decision Tree Classifier."
)

interface.launch()
